## Plots from DEG hits post Seurat + Metascape analysis

In [1]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import sys
import os
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns

# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

working_dir = "/ix/djishnu/Akanksha/snRNA_TA_muscle_analysis/Seurat_analysis_outputs/tables"

In [2]:
########## file paths ##########

#fast2b
female_fast2b_path = os.path.join(working_dir, "Female_Fast IIB_unfiltered_KO_DEGs_nmt.csv")
male_fast2b_path = os.path.join(working_dir, "Male_Fast IIB_unfiltered_KO_DEGs_nmt.csv")
#fast2x
female_fast2x_path = os.path.join(working_dir, "Female_Fast IIX_unfiltered_KO_DEGs_nmt.csv")
male_fast2x_path = os.path.join(working_dir, "Male_Fast IIX_unfiltered_KO_DEGs_nmt.csv")
#faps
female_faps_path = os.path.join(working_dir, "Female_FAPs_unfiltered_KO_DEGs_nmt.csv")
male_faps_path = os.path.join(working_dir, "Male_FAPs_unfiltered_KO_DEGs_nmt.csv")
#musc
female_musc_path = os.path.join(working_dir, "Female_Skeleton MuSc_unfiltered_KO_DEGs_nmt.csv")
male_musc_path = os.path.join(working_dir, "Male_Skeleton MuSc_unfiltered_KO_DEGs_nmt.csv")


#### Find common Up and Down DEGs in M and F per cell type

In [17]:
############ load DEGs ############
# load male and female DEG per cell type to memory in order to find common ros (merge pandas usins the common genes into another dataframe)
path_female_degs = female_fast2x_path
path_male_degs = male_fast2x_path
female_degs = pd.read_csv(path_female_degs, header=0)
male_degs = pd.read_csv(path_male_degs, header=0)
#convertlog2FC to ratio of fold change by exponentiating with base 2
female_degs['avg_FC'] = 2**female_degs['avg_log2FC']
male_degs['avg_FC'] = 2**male_degs['avg_log2FC']

#### P val Adj 0.05

In [18]:
#find common genes from matching gene names and merge the dataframes using those rows that have the common gene name
common_genes = pd.merge(female_degs, male_degs, on='Unnamed: 0', how='inner', suffixes=('_female', '_male'))
common_genes = common_genes.rename(columns={'Unnamed: 0':'gene_name'})
# Filter rows where the sign of 'avg_log2FC' matches (both positive or both negative)
filtered_degs = common_genes[((common_genes['avg_log2FC_female'] > 0) & (common_genes['avg_log2FC_male'] > 0)) | 
                                      ((common_genes['avg_log2FC_female'] < 0) & (common_genes['avg_log2FC_male'] < 0))]
#drop some columns
filtered_degs = filtered_degs.drop(columns = ['pct.1_female', 'pct.1_male', 'pct.2_female', 'pct.2_male', 'p_val_female', 'p_val_male'])
#only keep rows where pval_adj is less than 0.05 in both female and male columns
filtered_degs = filtered_degs[(filtered_degs['p_val_adj_female'] < 0.05) & (filtered_degs['p_val_adj_male'] < 0.05)]
print(filtered_degs.head())
print(filtered_degs.shape)

  gene_name  avg_log2FC_female  p_val_adj_female  avg_FC_female  \
1    Arrdc2           5.412563               0.0      42.593534   
3    Rilpl1           1.589999               0.0       3.010492   
5    Zbtb16           0.974756               0.0       1.965309   
6     Samd4           0.959707               0.0       1.944915   
7     Fgfr2           0.850036               0.0       1.802546   

   avg_log2FC_male  p_val_adj_male  avg_FC_male  
1         0.760835    2.658980e-04     1.694471  
3         0.426989    2.006844e-21     1.344425  
5         0.711094   1.532617e-105     1.637045  
6         0.324356    1.610652e-26     1.252105  
7         0.192023    5.024368e-17     1.142365  
(160, 7)


In [19]:
#split the df by if the avg_log2FC column is + or - and call the plus df as upregulated and minus df as downregulated
upregulated_common = filtered_degs[filtered_degs['avg_log2FC_female'] > 0]
downregulated_common = filtered_degs[filtered_degs['avg_log2FC_female'] < 0]   
print(upregulated_common.shape)
print(downregulated_common.shape) 

(104, 7)
(56, 7)


In [20]:
#write to csv called common_MF_DEGs
upregulated_common.to_csv(os.path.join(working_dir, "common_MF_DEGs_up_in_KO_Fast2x.csv"), index=False)
downregulated_common.to_csv(os.path.join(working_dir, "common_MF_DEGs_down_in_KO_Fast2x.csv"), index=False)

In [ ]:
# # export this pandas dataframe to google sheets pygsheets
# import pygsheets
# #authorization
# gc = pygsheets.authorize(service_file='/Users/Akanksha/JugnuLab/data_analysis/credentials.json')
# #open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
# sh = gc.open('DEGs_common_genes')
# #select the first sheet
# wks = sh[0]
# #update the first sheet with df, starting at cell B2.
# wks.set_dataframe(filtered_degs, 'B2')
# print("Dataframe successfully uploaded to google sheets")